# Preparación del entorno

In [16]:
!pip install wget

In [17]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

In [18]:
import wget

url = "https://storage.googleapis.com/open-courses/dmeyf2023-8a1e/competencia_03_crudo.csv.gz"
ruta_destino = "/content/competencia_03_crudo.csv.gz"

wget.download(url, ruta_destino)

print("Archivo descargado en:", ruta_destino)

Archivo descargado en: /content/competencia_03_crudo.csv.gz


# Prueba con SQL

In [19]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:////content/dmeyf.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [20]:
%%sql
create or replace table competencia_03 as
select
    *
from read_csv_auto('/content/competencia_03_crudo.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


## Queries Básicas

In [21]:
%sql select * from competencia_03 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,0.0,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,0.0,1,0.00,-67414.82,1149.57,3299,28503.06,10,0,1395.87
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,0.0,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,0.0,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,0.0,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66


In [22]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_03
group by foto_mes

,foto_mes,cantidad
0,201901,124864
1,201902,125898
2,201903,126536
3,201904,127059
4,201905,127683
5,201906,129181
6,201907,130551
7,201908,132615
8,201909,134267
9,201910,136515


In [23]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_03
), clientes as (
    select distinct numero_de_cliente from competencia_03
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , CASE
    WHEN (mes_0 = 1) AND (mes_1 = 1) AND (mes_2 = 1) THEN 'CONTINUA'
    WHEN (mes_0 = 1) AND (mes_1 = 0) AND (mes_2 = 1) THEN 'CONTINUA'
    WHEN (mes_0 = 1) AND (mes_1 = 0) AND (mes_2 = 0) THEN 'BAJA+1'
    WHEN (mes_0 = 1) AND (mes_1 = 1) AND (mes_2 = 0) THEN 'BAJA+2'
    ELSE NULL END AS clase_ternaria
    from todo t
    left join competencia_03 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria where mes_0 = 1

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [24]:
%%sql
alter table competencia_03 add column clase_ternaria VARCHAR(10)

,Success


In [25]:
%%sql
update competencia_03
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_03.numero_de_cliente = targets.numero_de_cliente and competencia_03.foto_mes = targets.foto_mes;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [26]:
%%sql
SELECT clase_ternaria, COUNT(*) AS count_clase
FROM targets
GROUP BY clase_ternaria;

,clase_ternaria,count_clase
0,None,330218
1,BAJA+2,21329
2,CONTINUA,4520714
3,BAJA+1,20767


In [27]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_03
group by foto_mes

,foto_mes,cantidad
0,201901,124864
1,201902,125898
2,201903,126536
3,201904,127059
4,201905,127683
5,201906,129181
6,201907,130551
7,201908,132615
8,201909,134267
9,201910,136515


# Lags 1, 3, 6

In [ ]:
%%sql
create or replace table competencia_03_lags as
select
  numero_de_cliente
  ,foto_mes
  ,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,mprestamos_hipotecarios,cplazo_fijo,mplazo_fijo_dolares,mplazo_fijo_pesos,cinversion1,minversion1_pesos,minversion1_dolares,cinversion2,minversion2,cseguro_vida,cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales,ccaja_seguridad,cpayroll_trx,mpayroll,mpayroll2,cpayroll2_trx,ccuenta_debitos_automaticos,mcuenta_debitos_automaticos,ctarjeta_visa_debitos_automaticos,mttarjeta_visa_debitos_automaticos,ctarjeta_master_debitos_automaticos,mttarjeta_master_debitos_automaticos,cpagodeservicios,mpagodeservicios,cpagomiscuentas,mpagomiscuentas,ccajeros_propios_descuentos,mcajeros_propios_descuentos,ctarjeta_visa_descuentos,mtarjeta_visa_descuentos,ctarjeta_master_descuentos,mtarjeta_master_descuentos,ccomisiones_mantenimiento,mcomisiones_mantenimiento,ccomisiones_otras,mcomisiones_otras,cforex,cforex_buy,mforex_buy,cforex_sell,mforex_sell,ctransferencias_recibidas,mtransferencias_recibidas,ctransferencias_emitidas,mtransferencias_emitidas,cextraccion_autoservicio,mextraccion_autoservicio,ccheques_depositados,mcheques_depositados,ccheques_emitidos,mcheques_emitidos,ccheques_depositados_rechazados,mcheques_depositados_rechazados,ccheques_emitidos_rechazados,mcheques_emitidos_rechazados,tcallcenter,ccallcenter_transacciones,thomebanking,chomebanking_transacciones,ccajas_transacciones,ccajas_consultas,ccajas_depositos,ccajas_extracciones,ccajas_otras,catm_trx,matm,catm_trx_other,matm_other,ctrx_quarter,Master_delinquency,Master_status,Master_mfinanciacion_limite,Master_Fvencimiento,Master_Finiciomora,Master_msaldototal,Master_msaldopesos,Master_msaldodolares,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_fultimo_cierre,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_Finiciomora,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
  ,lag(active_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_active_quarter
  ,lag(active_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_active_quarter
  ,lag(active_quarter, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_active_quarter

  ,lag(cliente_vip, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_vip
  ,lag(cliente_vip, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cliente_vip
  ,lag(cliente_vip, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cliente_vip

  ,lag(internet, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_internet
  ,lag(internet, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_internet
  ,lag(internet, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_internet

  ,lag(cliente_edad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_edad
  ,lag(cliente_edad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cliente_edad
  ,lag(cliente_edad, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cliente_edad

  ,lag(cliente_antiguedad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_antiguedad
  ,lag(cliente_antiguedad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cliente_antiguedad
  ,lag(cliente_antiguedad, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cliente_antiguedad

  ,lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad
  ,lag(mrentabilidad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mrentabilidad
  ,lag(mrentabilidad, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mrentabilidad

  ,lag(mrentabilidad_annual, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad_annual
  ,lag(mrentabilidad_annual, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mrentabilidad_annual
  ,lag(mrentabilidad_annual, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mrentabilidad_annual

  ,lag(mcomisiones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones
  ,lag(mcomisiones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones
  ,lag(mcomisiones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcomisiones

  ,lag(mactivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mactivos_margen
  ,lag(mactivos_margen, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mactivos_margen
  ,lag(mactivos_margen, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mactivos_margen

  ,lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen
  ,lag(mpasivos_margen, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpasivos_margen
  ,lag(mpasivos_margen, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpasivos_margen

  ,lag(cproductos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cproductos
  ,lag(cproductos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cproductos
  ,lag(cproductos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cproductos

  ,lag(tcuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tcuentas
  ,lag(tcuentas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_tcuentas
  ,lag(tcuentas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_tcuentas

  ,lag(ccuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccuenta_corriente
  ,lag(ccuenta_corriente, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccuenta_corriente
  ,lag(ccuenta_corriente, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccuenta_corriente

  ,lag(mcuenta_corriente_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente_adicional
  ,lag(mcuenta_corriente_adicional, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuenta_corriente_adicional
  ,lag(mcuenta_corriente_adicional, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcuenta_corriente_adicional

  ,lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente
  ,lag(mcuenta_corriente, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuenta_corriente
  ,lag(mcuenta_corriente, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcuenta_corriente

  ,lag(ccaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccaja_ahorro
  ,lag(ccaja_ahorro, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccaja_ahorro
  ,lag(ccaja_ahorro, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccaja_ahorro

  ,lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro
  ,lag(mcaja_ahorro, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro
  ,lag(mcaja_ahorro, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcaja_ahorro

  ,lag(mcaja_ahorro_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_adicional
  ,lag(mcaja_ahorro_adicional, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro_adicional
  ,lag(mcaja_ahorro_adicional, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcaja_ahorro_adicional

  ,lag(mcaja_ahorro_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_dolares
  ,lag(mcaja_ahorro_dolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro_dolares
  ,lag(mcaja_ahorro_dolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcaja_ahorro_dolares

  ,lag(cdescubierto_preacordado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cdescubierto_preacordado
  ,lag(cdescubierto_preacordado, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cdescubierto_preacordado
  ,lag(cdescubierto_preacordado, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cdescubierto_preacordado

  ,lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo
  ,lag(mcuentas_saldo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuentas_saldo
  ,lag(mcuentas_saldo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcuentas_saldo

  ,lag(ctarjeta_debito, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito
  ,lag(ctarjeta_debito, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_debito
  ,lag(ctarjeta_debito, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_debito

  ,lag(ctarjeta_debito_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito_transacciones
  ,lag(ctarjeta_debito_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_debito_transacciones
  ,lag(ctarjeta_debito_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_debito_transacciones

  ,lag(mautoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mautoservicio
  ,lag(mautoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mautoservicio
  ,lag(mautoservicio, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mautoservicio

  ,lag(ctarjeta_visa, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa
  ,lag(ctarjeta_visa, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa
  ,lag(ctarjeta_visa, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_visa

  ,lag(ctarjeta_visa_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_transacciones
  ,lag(ctarjeta_visa_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa_transacciones
  ,lag(ctarjeta_visa_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_visa_transacciones

  ,lag(mtarjeta_visa_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_consumo
  ,lag(mtarjeta_visa_consumo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_visa_consumo
  ,lag(mtarjeta_visa_consumo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtarjeta_visa_consumo

  ,lag(ctarjeta_master, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master
  ,lag(ctarjeta_master, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master
  ,lag(ctarjeta_master, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_master

  ,lag(ctarjeta_master_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_transacciones
  ,lag(ctarjeta_master_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master_transacciones
  ,lag(ctarjeta_master_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_master_transacciones

  ,lag(mtarjeta_master_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_consumo
  ,lag(mtarjeta_master_consumo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_master_consumo
  ,lag(mtarjeta_master_consumo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtarjeta_master_consumo

  ,lag(cprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_personales
  ,lag(cprestamos_personales, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cprestamos_personales
  ,lag(cprestamos_personales, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cprestamos_personales

  ,lag(mprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales
  ,lag(mprestamos_personales, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mprestamos_personales
  ,lag(mprestamos_personales, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mprestamos_personales

  ,lag(cprestamos_prendarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_prendarios
  ,lag(cprestamos_prendarios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cprestamos_prendarios
  ,lag(cprestamos_prendarios, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cprestamos_prendarios

  ,lag(mprestamos_prendarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_prendarios
  ,lag(mprestamos_prendarios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mprestamos_prendarios
  ,lag(mprestamos_prendarios, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mprestamos_prendarios

  ,lag(cprestamos_hipotecarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_hipotecarios
  ,lag(cprestamos_hipotecarios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cprestamos_hipotecarios
  ,lag(cprestamos_hipotecarios, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cprestamos_hipotecarios

  ,lag(mprestamos_hipotecarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_hipotecarios
  ,lag(mprestamos_hipotecarios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mprestamos_hipotecarios
  ,lag(mprestamos_hipotecarios, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mprestamos_hipotecarios

  ,lag(cplazo_fijo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cplazo_fijo
  ,lag(cplazo_fijo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cplazo_fijo
  ,lag(cplazo_fijo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cplazo_fijo

  ,lag(mplazo_fijo_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_dolares
  ,lag(mplazo_fijo_dolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mplazo_fijo_dolares
  ,lag(mplazo_fijo_dolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mplazo_fijo_dolares

  ,lag(mplazo_fijo_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_pesos
  ,lag(mplazo_fijo_pesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mplazo_fijo_pesos
  ,lag(mplazo_fijo_pesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mplazo_fijo_pesos

  ,lag(cinversion1, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cinversion1
  ,lag(cinversion1, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cinversion1
  ,lag(cinversion1, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cinversion1

  ,lag(minversion1_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_pesos
  ,lag(minversion1_pesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_minversion1_pesos
  ,lag(minversion1_pesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_minversion1_pesos

  ,lag(minversion1_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_dolares
  ,lag(minversion1_dolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_minversion1_dolares
  ,lag(minversion1_dolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_minversion1_dolares

  ,lag(cinversion2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cinversion2
  ,lag(cinversion2, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cinversion2
  ,lag(cinversion2, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cinversion2

  ,lag(minversion2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion2
  ,lag(minversion2, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_minversion2
  ,lag(minversion2, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_minversion2

  ,lag(cseguro_vida, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_vida
  ,lag(cseguro_vida, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_vida
  ,lag(cseguro_vida, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cseguro_vida

  ,lag(cseguro_auto, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_auto
  ,lag(cseguro_auto, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_auto
  ,lag(cseguro_auto, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cseguro_auto

  ,lag(cseguro_vivienda, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_vivienda
  ,lag(cseguro_vivienda, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_vivienda
  ,lag(cseguro_vivienda, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cseguro_vivienda

  ,lag(cseguro_accidentes_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_accidentes_personales
  ,lag(cseguro_accidentes_personales, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_accidentes_personales
  ,lag(cseguro_accidentes_personales, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cseguro_accidentes_personales

  ,lag(ccaja_seguridad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccaja_seguridad
  ,lag(ccaja_seguridad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccaja_seguridad
  ,lag(ccaja_seguridad, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccaja_seguridad

  ,lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
  ,lag(cpayroll_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpayroll_trx
  ,lag(cpayroll_trx, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cpayroll_trx

  ,lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll
  ,lag(mpayroll, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpayroll
  ,lag(mpayroll, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpayroll

  ,lag(mpayroll2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll2
  ,lag(mpayroll2, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpayroll2
  ,lag(mpayroll2, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpayroll2

  ,lag(cpayroll2_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll2_trx
  ,lag(cpayroll2_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpayroll2_trx
  ,lag(cpayroll2_trx, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cpayroll2_trx

  ,lag(ccuenta_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccuenta_debitos_automaticos
  ,lag(ccuenta_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccuenta_debitos_automaticos
  ,lag(ccuenta_debitos_automaticos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccuenta_debitos_automaticos

  ,lag(mcuenta_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_debitos_automaticos
  ,lag(mcuenta_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuenta_debitos_automaticos
  ,lag(mcuenta_debitos_automaticos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcuenta_debitos_automaticos

  ,lag(ctarjeta_visa_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_debitos_automaticos
  ,lag(ctarjeta_visa_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa_debitos_automaticos
  ,lag(ctarjeta_visa_debitos_automaticos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_visa_debitos_automaticos

  ,lag(mttarjeta_visa_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_visa_debitos_automaticos
  ,lag(mttarjeta_visa_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mttarjeta_visa_debitos_automaticos
  ,lag(mttarjeta_visa_debitos_automaticos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mttarjeta_visa_debitos_automaticos

  ,lag(ctarjeta_master_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_debitos_automaticos
  ,lag(ctarjeta_master_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master_debitos_automaticos
  ,lag(ctarjeta_master_debitos_automaticos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_master_debitos_automaticos

  ,lag(mttarjeta_master_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_master_debitos_automaticos
  ,lag(mttarjeta_master_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mttarjeta_master_debitos_automaticos
  ,lag(mttarjeta_master_debitos_automaticos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mttarjeta_master_debitos_automaticos

  ,lag(cpagodeservicios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpagodeservicios
  ,lag(cpagodeservicios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpagodeservicios
  ,lag(cpagodeservicios, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cpagodeservicios

  ,lag(mpagodeservicios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagodeservicios
  ,lag(mpagodeservicios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpagodeservicios
  ,lag(mpagodeservicios, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpagodeservicios

  ,lag(cpagomiscuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpagomiscuentas
  ,lag(cpagomiscuentas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpagomiscuentas
  ,lag(cpagomiscuentas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cpagomiscuentas

  ,lag(mpagomiscuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagomiscuentas
  ,lag(mpagomiscuentas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpagomiscuentas
  ,lag(mpagomiscuentas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpagomiscuentas

  ,lag(ccajeros_propios_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajeros_propios_descuentos
  ,lag(ccajeros_propios_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajeros_propios_descuentos
  ,lag(ccajeros_propios_descuentos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajeros_propios_descuentos

  ,lag(mcajeros_propios_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcajeros_propios_descuentos
  ,lag(mcajeros_propios_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcajeros_propios_descuentos
  ,lag(mcajeros_propios_descuentos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcajeros_propios_descuentos

  ,lag(ctarjeta_visa_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_descuentos
  ,lag(ctarjeta_visa_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa_descuentos
  ,lag(ctarjeta_visa_descuentos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_visa_descuentos

  ,lag(mtarjeta_visa_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_descuentos
  ,lag(mtarjeta_visa_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_visa_descuentos
  ,lag(mtarjeta_visa_descuentos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtarjeta_visa_descuentos

  ,lag(ctarjeta_master_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_descuentos
  ,lag(ctarjeta_master_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master_descuentos
  ,lag(ctarjeta_master_descuentos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_master_descuentos

  ,lag(mtarjeta_master_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_descuentos
  ,lag(mtarjeta_master_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_master_descuentos
  ,lag(mtarjeta_master_descuentos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtarjeta_master_descuentos

  ,lag(ccomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccomisiones_mantenimiento
  ,lag(ccomisiones_mantenimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccomisiones_mantenimiento
  ,lag(ccomisiones_mantenimiento, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccomisiones_mantenimiento

  ,lag(mcomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_mantenimiento
  ,lag(mcomisiones_mantenimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones_mantenimiento
  ,lag(mcomisiones_mantenimiento, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcomisiones_mantenimiento

  ,lag(ccomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccomisiones_otras
  ,lag(ccomisiones_otras, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccomisiones_otras
  ,lag(ccomisiones_otras, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccomisiones_otras

  ,lag(mcomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_otras
  ,lag(mcomisiones_otras, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones_otras
  ,lag(mcomisiones_otras, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcomisiones_otras

  ,lag(cforex, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex
  ,lag(cforex, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cforex
  ,lag(cforex, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cforex

  ,lag(cforex_buy, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex_buy
  ,lag(cforex_buy, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cforex_buy
  ,lag(cforex_buy, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cforex_buy

  ,lag(mforex_buy, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_buy
  ,lag(mforex_buy, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mforex_buy
  ,lag(mforex_buy, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mforex_buy

  ,lag(cforex_sell, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex_sell
  ,lag(cforex_sell, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cforex_sell
  ,lag(cforex_sell, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cforex_sell

  ,lag(mforex_sell, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_sell
  ,lag(mforex_sell, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mforex_sell
  ,lag(mforex_sell, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mforex_sell

  ,lag(ctransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctransferencias_recibidas
  ,lag(ctransferencias_recibidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctransferencias_recibidas
  ,lag(ctransferencias_recibidas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctransferencias_recibidas

  ,lag(mtransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_recibidas
  ,lag(mtransferencias_recibidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtransferencias_recibidas
  ,lag(mtransferencias_recibidas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtransferencias_recibidas

  ,lag(ctransferencias_emitidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctransferencias_emitidas
  ,lag(ctransferencias_emitidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctransferencias_emitidas
  ,lag(ctransferencias_emitidas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctransferencias_emitidas

  ,lag(mtransferencias_emitidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_emitidas
  ,lag(mtransferencias_emitidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtransferencias_emitidas
  ,lag(mtransferencias_emitidas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtransferencias_emitidas

  ,lag(cextraccion_autoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cextraccion_autoservicio
  ,lag(cextraccion_autoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cextraccion_autoservicio
  ,lag(cextraccion_autoservicio, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cextraccion_autoservicio

  ,lag(mextraccion_autoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mextraccion_autoservicio
  ,lag(mextraccion_autoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mextraccion_autoservicio
  ,lag(mextraccion_autoservicio, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mextraccion_autoservicio

  ,lag(ccheques_depositados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_depositados
  ,lag(ccheques_depositados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_depositados
  ,lag(ccheques_depositados, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccheques_depositados

  ,lag(mcheques_depositados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados
  ,lag(mcheques_depositados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_depositados
  ,lag(mcheques_depositados, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcheques_depositados

  ,lag(ccheques_emitidos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_emitidos
  ,lag(ccheques_emitidos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_emitidos
  ,lag(ccheques_emitidos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccheques_emitidos

  ,lag(mcheques_emitidos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos
  ,lag(mcheques_emitidos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_emitidos
  ,lag(mcheques_emitidos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcheques_emitidos

  ,lag(ccheques_depositados_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_depositados_rechazados
  ,lag(ccheques_depositados_rechazados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_depositados_rechazados
  ,lag(ccheques_depositados_rechazados, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccheques_depositados_rechazados

  ,lag(mcheques_depositados_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados_rechazados
  ,lag(mcheques_depositados_rechazados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_depositados_rechazados
  ,lag(mcheques_depositados_rechazados, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcheques_depositados_rechazados

  ,lag(ccheques_emitidos_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_emitidos_rechazados
  ,lag(ccheques_emitidos_rechazados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_emitidos_rechazados
  ,lag(ccheques_emitidos_rechazados, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccheques_emitidos_rechazados

  ,lag(mcheques_emitidos_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos_rechazados
  ,lag(mcheques_emitidos_rechazados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_emitidos_rechazados
  ,lag(mcheques_emitidos_rechazados, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcheques_emitidos_rechazados

  ,lag(tcallcenter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tcallcenter
  ,lag(tcallcenter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_tcallcenter
  ,lag(tcallcenter, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_tcallcenter

  ,lag(ccallcenter_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccallcenter_transacciones
  ,lag(ccallcenter_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccallcenter_transacciones
  ,lag(ccallcenter_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccallcenter_transacciones

  ,lag(thomebanking, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_thomebanking
  ,lag(thomebanking, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_thomebanking
  ,lag(thomebanking, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_thomebanking

  ,lag(chomebanking_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_chomebanking_transacciones
  ,lag(chomebanking_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_chomebanking_transacciones
  ,lag(chomebanking_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_chomebanking_transacciones

  ,lag(ccajas_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_transacciones
  ,lag(ccajas_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_transacciones
  ,lag(ccajas_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajas_transacciones

  ,lag(ccajas_consultas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_consultas
  ,lag(ccajas_consultas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_consultas
  ,lag(ccajas_consultas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajas_consultas

  ,lag(ccajas_depositos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_depositos
  ,lag(ccajas_depositos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_depositos
  ,lag(ccajas_depositos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajas_depositos

  ,lag(ccajas_extracciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_extracciones
  ,lag(ccajas_extracciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_extracciones
  ,lag(ccajas_extracciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajas_extracciones

  ,lag(ccajas_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_otras
  ,lag(ccajas_otras, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_otras
  ,lag(ccajas_otras, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajas_otras

  ,lag(catm_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_catm_trx
  ,lag(catm_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_catm_trx
  ,lag(catm_trx, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_catm_trx

  ,lag(matm, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm
  ,lag(matm, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_matm
  ,lag(matm, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_matm

  ,lag(catm_trx_other, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_catm_trx_other
  ,lag(catm_trx_other, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_catm_trx_other
  ,lag(catm_trx_other, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_catm_trx_other

  ,lag(matm_other, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm_other
  ,lag(matm_other, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_matm_other
  ,lag(matm_other, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_matm_other

  ,lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  ,lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
  ,lag(ctrx_quarter, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctrx_quarter

  ,lag(Master_delinquency, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_delinquency
  ,lag(Master_delinquency, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_delinquency
  ,lag(Master_delinquency, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_delinquency

  ,lag(Master_status, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_status
  ,lag(Master_status, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_status
  ,lag(Master_status, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_status

  ,lag(Master_mfinanciacion_limite, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mfinanciacion_limite
  ,lag(Master_mfinanciacion_limite, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mfinanciacion_limite
  ,lag(Master_mfinanciacion_limite, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mfinanciacion_limite

  ,lag(Master_Fvencimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_Fvencimiento
  ,lag(Master_Fvencimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_Fvencimiento
  ,lag(Master_Fvencimiento, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_Fvencimiento

  ,lag(Master_Finiciomora, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_Finiciomora
  ,lag(Master_Finiciomora, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_Finiciomora
  ,lag(Master_Finiciomora, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_Finiciomora

  ,lag(Master_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldototal
  ,lag(Master_msaldototal, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_msaldototal
  ,lag(Master_msaldototal, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_msaldototal

  ,lag(Master_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldopesos
  ,lag(Master_msaldopesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_msaldopesos
  ,lag(Master_msaldopesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_msaldopesos

  ,lag(Master_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldodolares
  ,lag(Master_msaldodolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_msaldodolares
  ,lag(Master_msaldodolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_msaldodolares

  ,lag(Master_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumospesos
  ,lag(Master_mconsumospesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mconsumospesos
  ,lag(Master_mconsumospesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mconsumospesos

  ,lag(Master_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumosdolares
  ,lag(Master_mconsumosdolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mconsumosdolares
  ,lag(Master_mconsumosdolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mconsumosdolares

  ,lag(Master_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mlimitecompra
  ,lag(Master_mlimitecompra, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mlimitecompra
  ,lag(Master_mlimitecompra, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mlimitecompra

  ,lag(Master_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantopesos
  ,lag(Master_madelantopesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_madelantopesos
  ,lag(Master_madelantopesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_madelantopesos

  ,lag(Master_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantodolares
  ,lag(Master_madelantodolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_madelantodolares
  ,lag(Master_madelantodolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_madelantodolares

  ,lag(Master_fultimo_cierre, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_fultimo_cierre
  ,lag(Master_fultimo_cierre, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_fultimo_cierre
  ,lag(Master_fultimo_cierre, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_fultimo_cierre

  ,lag(Master_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagado
  ,lag(Master_mpagado, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagado
  ,lag(Master_mpagado, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mpagado

  ,lag(Master_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagospesos
  ,lag(Master_mpagospesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagospesos
  ,lag(Master_mpagospesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mpagospesos

  ,lag(Master_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagosdolares
  ,lag(Master_mpagosdolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagosdolares
  ,lag(Master_mpagosdolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mpagosdolares

  ,lag(Master_fechaalta, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_fechaalta
  ,lag(Master_fechaalta, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_fechaalta
  ,lag(Master_fechaalta, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_fechaalta

  ,lag(Master_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumototal
  ,lag(Master_mconsumototal, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mconsumototal
  ,lag(Master_mconsumototal, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mconsumototal

  ,lag(Master_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cconsumos
  ,lag(Master_cconsumos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_cconsumos
  ,lag(Master_cconsumos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_cconsumos

  ,lag(Master_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cadelantosefectivo
  ,lag(Master_cadelantosefectivo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_cadelantosefectivo
  ,lag(Master_cadelantosefectivo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_cadelantosefectivo

  ,lag(Master_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagominimo
  ,lag(Master_mpagominimo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagominimo
  ,lag(Master_mpagominimo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mpagominimo

  ,lag(Visa_delinquency, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_delinquency
  ,lag(Visa_delinquency, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_delinquency
  ,lag(Visa_delinquency, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_delinquency

  ,lag(Visa_status, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_status
  ,lag(Visa_status, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_status
  ,lag(Visa_status, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_status

  ,lag(Visa_mfinanciacion_limite, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mfinanciacion_limite
  ,lag(Visa_mfinanciacion_limite, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mfinanciacion_limite
  ,lag(Visa_mfinanciacion_limite, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mfinanciacion_limite

  ,lag(Visa_Fvencimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Fvencimiento
  ,lag(Visa_Fvencimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_Fvencimiento
  ,lag(Visa_Fvencimiento, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_Fvencimiento

  ,lag(Visa_Finiciomora, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Finiciomora
  ,lag(Visa_Finiciomora, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_Finiciomora
  ,lag(Visa_Finiciomora, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_Finiciomora

  ,lag(Visa_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldototal
  ,lag(Visa_msaldototal, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_msaldototal
  ,lag(Visa_msaldototal, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_msaldototal

  ,lag(Visa_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldopesos
  ,lag(Visa_msaldopesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_msaldopesos
  ,lag(Visa_msaldopesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_msaldopesos

  ,lag(Visa_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldodolares
  ,lag(Visa_msaldodolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_msaldodolares
  ,lag(Visa_msaldodolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_msaldodolares

  ,lag(Visa_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumospesos
  ,lag(Visa_mconsumospesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mconsumospesos
  ,lag(Visa_mconsumospesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mconsumospesos

  ,lag(Visa_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumosdolares
  ,lag(Visa_mconsumosdolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mconsumosdolares
  ,lag(Visa_mconsumosdolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mconsumosdolares

  ,lag(Visa_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mlimitecompra
  ,lag(Visa_mlimitecompra, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mlimitecompra
  ,lag(Visa_mlimitecompra, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mlimitecompra

  ,lag(Visa_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantopesos
  ,lag(Visa_madelantopesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_madelantopesos
  ,lag(Visa_madelantopesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_madelantopesos

  ,lag(Visa_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantodolares
  ,lag(Visa_madelantodolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_madelantodolares
  ,lag(Visa_madelantodolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_madelantodolares

  ,lag(Visa_fultimo_cierre, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_fultimo_cierre
  ,lag(Visa_fultimo_cierre, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_fultimo_cierre
  ,lag(Visa_fultimo_cierre, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_fultimo_cierre

  ,lag(Visa_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagado
  ,lag(Visa_mpagado, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagado
  ,lag(Visa_mpagado, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mpagado

  ,lag(Visa_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagospesos
  ,lag(Visa_mpagospesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagospesos
  ,lag(Visa_mpagospesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mpagospesos

  ,lag(Visa_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagosdolares
  ,lag(Visa_mpagosdolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagosdolares
  ,lag(Visa_mpagosdolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mpagosdolares

  ,lag(Visa_fechaalta, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_fechaalta
  ,lag(Visa_fechaalta, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_fechaalta
  ,lag(Visa_fechaalta, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_fechaalta

  ,lag(Visa_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumototal
  ,lag(Visa_mconsumototal, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mconsumototal
  ,lag(Visa_mconsumototal, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mconsumototal

  ,lag(Visa_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cconsumos
  ,lag(Visa_cconsumos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_cconsumos
  ,lag(Visa_cconsumos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_cconsumos

  ,lag(Visa_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cadelantosefectivo
  ,lag(Visa_cadelantosefectivo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_cadelantosefectivo
  ,lag(Visa_cadelantosefectivo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_cadelantosefectivo

  ,lag(Visa_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagominimo
  ,lag(Visa_mpagominimo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagominimo
  ,lag(Visa_mpagominimo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mpagominimo

from competencia_03

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
%%sql
copy competencia_03_lags to '/content/competencia_03_base_lags.csv.gz' (FORMAT CSV, HEADER)